#### Run ROI-level analyses 
    - plot unphased and phased timeseries 
    - plot frequency spectrum
    - reliability metrics
sbatch --time=10:00:00 --cpus-per-task=2 --mem-per-cpu=8GB --account=def-mmur {script}

In [1]:

from pathlib import Path
import itertools

FO_THR = .8
PCORR = "fdrp" # fdrp or uncp
smooth_id = 0
n_iterations = 25
n_batches_expected = 16
time_window = (39, 219)
TR = .3
base_dir = "/scratch/fastfmri"

In [2]:

experiment_id = "1_frequency_tagging"
mri_id = "7T"
# Iterate over these options
desc_ids = ["basic"]*6
roi_task_ids = ["entrainD"]*2 + ["entrainE"]*2 + ["entrainF"]*2
roi_task_frequencies = [.125, .2] + [.15, .2] + [.175,.2]
task_conditions_and_search_frequencies = [
    ("entrainD", [.125, .2]), 
    ("entrainE", [.15, .2]), 
    ("entrainF", [.175, .2]),
]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["021"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)

[basic] sub-021, roi: entrainD [0.125], task: entrainD [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainD-0.125_task-entrainD_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-021, roi: entrainD [0.125], task: entrainE [0.15, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainD-0.125_task-entrainE_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-021, roi: entrainD [0.125], task: entrainF [0.175, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainD-0.125_task-entrainF_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-021, roi: entrainD [0.2], task: entrainD [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basi

In [3]:

experiment_id = "1_frequency_tagging"
mri_id = "7T"
# Iterate over these options
desc_ids = ["basic"]*6
roi_task_ids = ["entrainA"]*2 + ["entrainB"]*2 + ["entrainC"]*2
roi_task_frequencies = [.125, .2] + [.125, .175] + [.125,.15]
task_conditions_and_search_frequencies = [
    ("entrainA", [.125, .2]), 
    ("entrainB", [.125, .175]), 
    ("entrainC", [.125, .15]),
]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["020"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)

[basic] sub-020, roi: entrainA [0.125], task: entrainA [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainA-0.125_task-entrainA_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-020, roi: entrainA [0.125], task: entrainB [0.125, 0.175]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainA-0.125_task-entrainB_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-020, roi: entrainA [0.125], task: entrainC [0.125, 0.15]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainA-0.125_task-entrainC_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-020, roi: entrainA [0.2], task: entrainA [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-7T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-

In [4]:

experiment_id = "1_frequency_tagging"
mri_id = "3T"
# Iterate over these options
desc_ids = ["basic"]*6
roi_task_ids = ["entrainD"]*2 + ["entrainE"]*2 + ["entrainF"]*2
roi_task_frequencies = [.125, .2] + [.15, .2] + [.175,.2]
task_conditions_and_search_frequencies = [
    ("entrainD", [.125, .2]), 
    ("entrainE", [.15, .2]), 
    ("entrainF", [.175, .2]),
]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["021"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)

[basic] sub-021, roi: entrainD [0.125], task: entrainD [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainD-0.125_task-entrainD_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-021, roi: entrainD [0.125], task: entrainE [0.15, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainD-0.125_task-entrainE_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-021, roi: entrainD [0.125], task: entrainF [0.175, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainD-0.125_task-entrainF_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-021, roi: entrainD [0.2], task: entrainD [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-021_smooth-0_truncate-39-219_n-400_batch-merged_desc-basi

In [5]:

experiment_id = "1_frequency_tagging"
mri_id = "3T"
# Iterate over these options
desc_ids = ["basic"]*6
roi_task_ids = ["entrainA"]*2 + ["entrainB"]*2 + ["entrainC"]*2
roi_task_frequencies = [.125, .2] + [.125, .175] + [.125,.15]
task_conditions_and_search_frequencies = [
    ("entrainA", [.125, .2]), 
    ("entrainB", [.125, .175]), 
    ("entrainC", [.125, .15]),
]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["020"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)


[basic] sub-020, roi: entrainA [0.125], task: entrainA [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainA-0.125_task-entrainA_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-020, roi: entrainA [0.125], task: entrainB [0.125, 0.175]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainA-0.125_task-entrainB_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-020, roi: entrainA [0.125], task: entrainC [0.125, 0.15]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrainA-0.125_task-entrainC_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-020, roi: entrainA [0.2], task: entrainA [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-020_smooth-0_truncate-39-219_n-400_batch-merged_desc-

In [6]:

experiment_id = "1_frequency_tagging"
mri_id = "3T"
# Iterate over these options
desc_ids = ["basic"]*4
roi_task_ids = ["entrain"]*2 + ["control"]*2
roi_task_frequencies = [.125, .2]*2
task_conditions_and_search_frequencies = [
    ("control", [.125, .2]), 
    ("entrain", [.125, .2]), 
]*2

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["000", "002", "003", "004", "005", "006", "007", "008", "009"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)


[basic] sub-000, roi: entrain [0.125], task: control [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrain-0.125_task-control_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-000, roi: entrain [0.125], task: entrain [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrain-0.125_task-entrain_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-000, roi: entrain [0.125], task: control [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entrain-0.125_task-control_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-000, roi: entrain [0.125], task: entrain [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_frequency_tagging_mri-3T_sub-000_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-entra

In [7]:
experiment_id = "1_attention"
mri_id = "7T"
# Iterate over these options
desc_ids = ["basic"]*2
roi_task_ids = ["AttendAway"]*2
roi_task_frequencies = [.125, .2]
task_conditions_and_search_frequencies = [("AttendAway", [.125, .2]),]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["Pilot001", "Pilot009", "Pilot010", "Pilot011"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]
        
        if roi_task_id != task_condition:
            continue
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)


[basic] sub-Pilot001, roi: AttendAway [0.125], task: AttendAway [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_attention_mri-7T_sub-Pilot001_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-AttendAway-0.125_task-AttendAway_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-Pilot001, roi: AttendAway [0.2], task: AttendAway [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_attention_mri-7T_sub-Pilot001_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-AttendAway-0.2_task-AttendAway_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-Pilot009, roi: AttendAway [0.125], task: AttendAway [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_attention_mri-7T_sub-Pilot009_smooth-0_truncate-39-219_n-400_batch-merged_desc-basic_roi-AttendAway-0.125_task-AttendAway_pval-fdrp_fo-0.8_bootstrap.sh
[basic] sub-Pilot009, roi: AttendAway [0.2], task: AttendAway [0.125, 0.2]
/data/scripts/03c_merge_bootstraps/experiment-1_attention_mri-7T_sub-Pilot009_smooth-0_truncate-39

### Not part of the frequency tagging study

In [ ]:

experiment_id = "1_attention"
mri_id = "7T"
smooth_id = 0
n_iterations = 100
n_batches_expected = 2
time_window = (39, 219)
TR = .25
base_dir = "/scratch/fastfmri"
# Iterate over these options
desc_ids = ["basic"]*8
roi_task_ids = ["AttendAway"]*2 + ["AttendInF1"]*2 + ["AttendInF2"]*2 + ["AttendInF1F2"]*2
roi_task_frequencies = [.125, .2]*4
task_conditions_and_search_frequencies = [
    ("AttendAway", [.125, .2]), 
    ("AttendInF1", [.125, .2]), 
    ("AttendInF2", [.125, .2]), 
    ("AttendInF1F2", [.125, .2]), 
]

CONTAINER = '/project/def-mmur/gngo4/containers/neuroimaging-notebook-v2.simg'

# Output sbatch scripts here
scripts_dir = "/data/scripts/03c_merge_bootstraps"
scripts_dir = Path(scripts_dir)
if not scripts_dir.exists():
    scripts_dir.mkdir(parents=True)


for sub_id in ["010", "011", "012", "013", "014", "015", "016"]:

    for (desc_id, roi_task_id, roi_task_frequency), _task_condition_and_search_frequencies in itertools.product(
        zip(desc_ids, roi_task_ids, roi_task_frequencies),
        task_conditions_and_search_frequencies
    ):

        task_condition = _task_condition_and_search_frequencies[0]
        search_frequencies = _task_condition_and_search_frequencies[1]

        if roi_task_id != task_condition:
            continue
        
        print(f"[{desc_id}] sub-{sub_id}, roi: {roi_task_id} [{roi_task_frequency}], task: {task_condition} {search_frequencies}")

        # Check, and run if it doesn't exist
        out_dir_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap"
        )
        out_file = Path(base_dir) / out_dir_base / f"sub-{sub_id}" / "bootstrap" / f"task-{task_condition}_bootstrapped_data.pkl"
        if out_file.exists():
            continue

        txt = f"""#!/bin/bash
module load apptainer/1.2.4

singularity run \\
--bind /project/def-mmur/gngo4/data/fastfmri:/data \\
--bind /scratch/gngo4:/scratch \\
--bind /project/def-mmur/gngo4/projects/fastfmri_toolbox:/opt/app \\
{CONTAINER} \\
python3 /opt/app/scripts/concat_bootstrap.py \\
    --experiment-id {experiment_id} \\
    --mri-id {mri_id} \\
    --sub-id {sub_id} \\
    --smooth-mm {smooth_id} \\
    --desc-id {desc_id} \\
    --n-iterations {n_iterations} \\
    --n-batches-expected {n_batches_expected} \\
    --roi-task-id {roi_task_id} \\
    --roi-task-frequency {roi_task_frequency} \\
    --task-conditions {task_condition} \\
    --fractional-overlap-threshold {FO_THR} \\
    --search-frequencies {' --search-frequencies '.join([str(i) for i in search_frequencies])} \\
    --time-window {' '.join([str(i) for i in time_window])} \\
    --tr {TR} \\
    --pcorr {PCORR} \\
    --base-dir {base_dir}
        """
        out_base = (
            f"experiment-{experiment_id}"
            f"_mri-{mri_id}_sub-{sub_id}_smooth-{smooth_id}_truncate-{time_window[0]}-{time_window[1]}"
            f"_n-{n_iterations*n_batches_expected}_batch-merged_desc-{desc_id}_roi-{roi_task_id}-{roi_task_frequency}_task-{task_condition}_pval-{PCORR}_fo-{float(FO_THR)}_bootstrap.sh"
        )
        cmd_path = scripts_dir / out_base
        print(cmd_path)
        with open(cmd_path, 'w') as f:
            f.write(txt)
